In [1]:
import numpy as np
import math

The matrix reflects: in the rows, the topics. in the columns, the users (+ the last column, which is the Home column). The numbers in the matrix are the number of news of topic i that user j has encountered on the homepage.

In [4]:
americanNews_matrix = np.array([[1, 2, 1, 0, 0, 0, 0, 0, 0],
                                [2, 1, 3, 1, 0, 0, 0, 0, 4],
                                [0, 0, 2, 3, 0, 0, 0, 0, 2],
                                [0, 0, 0, 0, 6, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 8, 0, 0, 0],
                                [0, 0, 0, 0, 0, 1, 8, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 3, 0],
                                [0, 0, 0, 0, 0, 0, 0, 5, 0],
                                [7, 0, 0, 0, 0, 0, 0, 0, 0],
                                [0, 7, 0, 0, 0, 0, 0, 0, 0],
                                [0, 0, 4, 2, 0, 1, 0, 2, 0],
                                [0, 0, 0, 4, 0, 0, 0, 0, 0],
                                [0, 0, 0, 0, 4, 0, 0, 0, 0],
                                [0, 0, 0, 0, 0, 0, 2, 0, 0],
                                [0, 0, 0, 0, 0, 0, 0, 0, 2]])

In [5]:
indianNews_matrix = np.array([[3, 3, 1, 0, 2, 3, 3, 0, 0, 0, 1],
                              [1, 2, 4, 4, 1, 0, 0, 0, 0, 0, 4],
                              [2, 1, 1, 0, 2, 5, 3, 0, 0, 0, 2],
                              [4, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0],
                              [0, 4, 0, 6, 1, 0, 0, 10, 0, 0, 0],
                              [0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 3],
                              [0, 0, 0, 0, 0, 0, 0, 0, 10, 2, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0]])

### Click-through rate (CTR)

CTR (Click-Through Rate) is a metric that measures **the percentage of clicks on recommended articles compared to the total number of recommended articles**.

- Original CTR: In the original paper, the CTR (Click-Through Rate) is calculated as **the fraction of recommended articles (N) that the user clicks on (ri = 1)**. **The sum of clicks on recommended items is divided by the total number of recommended items (N items)**.

- Our CTR: *calculate_custom_CTR* calculates a CTR-like metric based on the matrix. The metric represents **the fraction of views in the "Home" column compared to total views**. This metric takes into account article views on the homepage.

In the original paper, CTR is defined as the fraction of recommended articles that the user clicks on, which represents specific user behavior towards recommended content.

In our case, we are trying to similarly evaluate how much users view content from a certain topic on the homepage, but without considering actual clicks. So, while the approach is similar in that they both seek to measure user interaction with recommended content, there are key differences in defining the metrics.

In [4]:
# Definisci una funzione per calcolare il CTR
def calculate_CTR(matrix):
    # Calcola la somma delle visualizzazioni nella matrice
    total_views = np.sum(matrix)
    
    # Calcola la somma delle visualizzazioni nella colonna "Home" (ultima colonna)
    home_views = np.sum(matrix[:, -1])
    
    # Calcola la metrica come la frazione di visualizzazioni nella colonna "Home"
    CTR = home_views / total_views
    
    return CTR

# Calcola il CTR
ctr = calculate_CTR(matrix)
print("Click-Through Rate (CTR):", ctr)

Click-Through Rate (CTR): 0.09090909090909091


Our Click-Through Rate (CTR) result is 0.09090909090909091, which represents the percentage of homepage article views compared to total views. In other words, approximately 8.33% of homepage views had a click or similar interaction.

### Average document stance

- Original Average document stance is the **average partisan score of the articles that are shown to the users**.

- Our Average document stance: *calculate_average_document_stance* takes **the topic array and index as input and calculates the average document stance for that specific topic**. This function calculates the average of the partisan scores for the topic, excluding zero values in the average.

In [6]:
def calculate_average_document_stance(matrix, topic_index):
    # Estrai la riga corrispondente al topic
    topic_row = matrix[topic_index - 1]  # Sottrai 1 perché gli indici partono da 0
    
    # Calcola la somma dei punteggi per il topic
    total_score = np.sum(topic_row)
    
    # Calcola l'average document stance
    avg_stance = total_score / np.sum(topic_row != 0)  # Calcola la media solo sui valori non zero
    
    return avg_stance

# Calcola l'average document stance per i tre topic: repubblicano, democratico, neutrale
avg_stance_repubblicano = calculate_average_document_stance(matrix, 1)  # Topic repubblicano
avg_stance_democratico = calculate_average_document_stance(matrix, 2)   # Topic democratico
avg_stance_neutrale = calculate_average_document_stance(matrix, 3)      # Topic neutrale

print("Average Document Stance per il topic Repubblicano:", avg_stance_repubblicano)
print("Average Document Stance per il topic Democratico:", avg_stance_democratico)
print("Average Document Stance per il topic Neutrale:", avg_stance_neutrale)

Average Document Stance per il topic Repubblicano: 1.3333333333333333
Average Document Stance per il topic Democratico: 2.2
Average Document Stance per il topic Neutrale: 2.3333333333333335


These results represent the average position of articles shown to users for each topic. For example, for the Republican topic, the average document stance is approximately 1.33, while for the Democratic topic it is approximately 2.2, and for the neutral topic it is approximately 2.33. These scores indicate the average positions of recommended articles in each topic, but remember that these values are specific to your display matrix and may vary based on actual data and user preferences.

### Average news document stance 

Average News Document Stance

U1 can have 3 stances
Republic -  normalize (U1(repub) 3, summation (all users republican)) 
Output u1 stance wrt republic
Democratic normalize (U1(demo) , summation (all users demo))
U1 stance wrt demo
Neutral normalize (U1(neu) , summation (all users neu))
U1 stance wrt republic

Repeat for all users

In [13]:
def calculate_american_scores(matrix):
    # Estrai le prime tre righe (republican, democratic, neutral)
    topics = matrix[:3]

    # Inizializza una lista per gli utenti
    users = []

    # Calcola i punteggi per ciascun utente
    for user_index in range(matrix.shape[1]):
        user = {}
        user['User'] = user_index + 1  # Per l'utente n, inizia da 1
        for topic_index, topic in enumerate(topics):
            topic_name = ''
            if topic_index == 0:
                topic_name = 'republican'
            elif topic_index == 1:
                topic_name = 'democratic'
            else:
                topic_name = 'neutral'
            
            user[f'{topic_name} score'] = topic[user_index] / sum(topic)
        
        users.append(user)

    return users

In [14]:
americanUser_scores = calculate_american_scores(americanNews_matrix)

# Stampa i risultati
for user in americanUser_scores:
    print(f"U{user['User']}:")
    for topic in ['republican', 'democratic', 'neutral']:
        print(f"{topic} score: {user[f'{topic} score']:.2f}")

U1:
republican score: 0.25
democratic score: 0.18
neutral score: 0.00
U2:
republican score: 0.50
democratic score: 0.09
neutral score: 0.00
U3:
republican score: 0.25
democratic score: 0.27
neutral score: 0.29
U4:
republican score: 0.00
democratic score: 0.09
neutral score: 0.43
U5:
republican score: 0.00
democratic score: 0.00
neutral score: 0.00
U6:
republican score: 0.00
democratic score: 0.00
neutral score: 0.00
U7:
republican score: 0.00
democratic score: 0.00
neutral score: 0.00
U8:
republican score: 0.00
democratic score: 0.00
neutral score: 0.00
U9:
republican score: 0.00
democratic score: 0.36
neutral score: 0.29


In [16]:
def calculate_indian_scores(matrix):
    # Estrai le prime tre righe (republican, democratic, neutral)
    topics = matrix[:3]

    # Inizializza una lista per gli utenti
    users = []

    # Calcola i punteggi per ciascun utente
    for user_index in range(matrix.shape[1]):
        user = {}
        user['User'] = user_index + 1  # Per l'utente n, inizia da 1
        for topic_index, topic in enumerate(topics):
            topic_name = ''
            if topic_index == 0:
                topic_name = 'progov'
            elif topic_index == 1:
                topic_name = 'proopp'
            else:
                topic_name = 'neutral'
            
            user[f'{topic_name} score'] = topic[user_index] / sum(topic)
        
        users.append(user)

    return users

In [17]:
indianUser_scores = calculate_indian_scores(indianNews_matrix)

# Stampa i risultati con nomi utente personalizzati
for user_index, user in enumerate(indianUser_scores, start=11):
    user_name = f"I{user_index}"
    print(f"{user_name}:")
    for topic in ['progov', 'proopp', 'neutral']:
        print(f"{topic} score: {user[f'{topic} score']:.2f}")


I11:
progov score: 0.19
proopp score: 0.06
neutral score: 0.12
I12:
progov score: 0.19
proopp score: 0.12
neutral score: 0.06
I13:
progov score: 0.06
proopp score: 0.25
neutral score: 0.06
I14:
progov score: 0.00
proopp score: 0.25
neutral score: 0.00
I15:
progov score: 0.12
proopp score: 0.06
neutral score: 0.12
I16:
progov score: 0.19
proopp score: 0.00
neutral score: 0.31
I17:
progov score: 0.19
proopp score: 0.00
neutral score: 0.19
I18:
progov score: 0.00
proopp score: 0.00
neutral score: 0.00
I19:
progov score: 0.00
proopp score: 0.00
neutral score: 0.00
I20:
progov score: 0.00
proopp score: 0.00
neutral score: 0.00
I21:
progov score: 0.06
proopp score: 0.25
neutral score: 0.12


### Normalized document stance

- Original Normalized document stance: they represent the fraction of articles that are shown to the users who have stance i. Normalized stance entropy is the entropy of this distribution,
normalized by log m so that its maximum is 1, where m = 5 in their case (representing the five political stances).

- Our Normalized document stance: *calculate_normalized_stance_entropy* takes the matrix and the value of m (in our case, 3) as input and **calculates the normalized position entropy based on the fractions of articles shown to users for each of the three positions**.

In [6]:
# Definisci una funzione per calcolare l'entropia di posizione normalizzata
def calculate_normalized_stance_entropy(matrix, m):
    # Calcola le frazioni di articoli mostrati agli utenti per ciascuna posizione
    position_fractions = np.sum(matrix, axis=1) / np.sum(matrix)
    
    # Calcola l'entropia della distribuzione
    entropy = -np.sum(position_fractions * np.log(position_fractions)) / np.log(m)
    
    return entropy

# Valore di m (numero di posizioni)
m = 3

# Calcola l'entropia di posizione normalizzata con m = 3
normalized_stance_entropy = calculate_normalized_stance_entropy(matrix, m)
print("Normalized Stance Entropy:", normalized_stance_entropy)

Normalized Stance Entropy: 2.369491306537947


This metric represents the diversity of item locations recommended to users. A higher entropy value indicates greater diversity in the positions of recommended items. In this case, a value of 2.369491306537947 suggests some diversity in item positions.

### Entropy (new)

for the entropy based 3 features, just generate for every user.

In [18]:
def calculate_american_user_entropy(matrix):
    users = []

    # iterate through columns
    for user_index in range(matrix.shape[1]):
        user = {}
        user['User'] = user_index + 1  

        # extracts value in row 0, 1 and 2 of user_index column
        rep = matrix[0, user_index]
        dem = matrix[1, user_index]
        neu = matrix[2, user_index]

        total = rep + dem + neu
        if total == 0:
            entropy = 0  # avoid division by zero
        else:
            rep_frac = rep / total
            dem_frac = dem / total
            neu_frac = neu / total

            if(rep_frac != 0):
                rep_var = -rep_frac * np.log(rep_frac)
            else:
                rep_var = 0
            if(dem_frac != 0): 
                dem_var = - dem_frac * np.log(dem_frac)
            else:
                dem_var = 0
            if(neu_frac != 0):  
                neu_var = - neu_frac * np.log(neu_frac)
            else:
                neu_var = 0
                
            if(not(math.isnan(rep_var)) and not(math.isnan(dem_var)) and not(math.isnan(neu_var))):
                entropy = rep_var + dem_var + neu_var

        user['rep'] = rep_var
        user['dem'] = dem_var
        user['neu'] = neu_var
        users.append(user)

    return users

In [22]:
american_user_entropies = calculate_american_user_entropy(americanNews_matrix)

for user in american_user_entropies:
    print(f"Rep for User U{user['User']}: {user['rep']:.2f}")
    print(f"Dem for User U{user['User']}: {user['dem']:.2f}")
    print(f"Neu for User U{user['User']}: {user['neu']:.2f}")
    print("==========================================")

Rep for User U1: 0.37
Dem for User U1: 0.27
Neu for User U1: 0.00
Rep for User U2: 0.27
Dem for User U2: 0.37
Neu for User U2: 0.00
Rep for User U3: 0.30
Dem for User U3: 0.35
Neu for User U3: 0.37
Rep for User U4: 0.00
Dem for User U4: 0.35
Neu for User U4: 0.22
Rep for User U5: 0.00
Dem for User U5: 0.35
Neu for User U5: 0.22
Rep for User U6: 0.00
Dem for User U6: 0.35
Neu for User U6: 0.22
Rep for User U7: 0.00
Dem for User U7: 0.35
Neu for User U7: 0.22
Rep for User U8: 0.00
Dem for User U8: 0.35
Neu for User U8: 0.22
Rep for User U9: 0.00
Dem for User U9: 0.27
Neu for User U9: 0.37


In [20]:
def calculate_indian_user_entropy(matrix):
    users = []

    # iterate through columns
    for user_index in range(matrix.shape[1]):
        user = {}
        user['User'] = user_index + 1  

        # extracts value in row 0, 1 and 2 of user_index column
        gov = matrix[0, user_index]
        opp = matrix[1, user_index]
        neu = matrix[2, user_index]

        total = gov + opp + neu
        if total == 0:
            entropy = 0  # avoid division by zero
        else:
            gov_frac = gov / total
            opp_frac = opp / total
            neu_frac = neu / total

            if(gov_frac != 0):
                gov_var = -gov_frac * np.log(gov_frac)
            else:
                gov_var = 0
            if(opp_frac != 0): 
                opp_var = - opp_frac * np.log(opp_frac)
            else:
                opp_var = 0
            if(neu_frac != 0):  
                neu_var = - neu_frac * np.log(neu_frac)
            else:
                neu_var = 0
                
            if(not(math.isnan(gov_var)) and not(math.isnan(opp_var)) and not(math.isnan(neu_var))):
                entropy = gov_var + opp_var + neu_var

        user['gov'] = gov_var
        user['opp'] = opp_var
        user['neu'] = neu_var
        users.append(user)

    return users

In [25]:
indian_user_entropies = calculate_indian_user_entropy(indianNews_matrix)

# Stampa i risultati con nomi utente personalizzati
for user_index, user in enumerate(indian_user_entropies, start=11):
    user_name = f"I{user_index}"
    print(f"Rep for User {user_name}: {user['gov']:.2f}")
    print(f"Dem for User {user_name}: {user['opp']:.2f}")
    print(f"Neu for User {user_name}: {user['neu']:.2f}")
    print("==========================================")


Rep for User I11: 0.35
Dem for User I11: 0.30
Neu for User I11: 0.37
Rep for User I12: 0.35
Dem for User I12: 0.37
Neu for User I12: 0.30
Rep for User I13: 0.30
Dem for User I13: 0.27
Neu for User I13: 0.30
Rep for User I14: 0.00
Dem for User I14: -0.00
Neu for User I14: 0.00
Rep for User I15: 0.37
Dem for User I15: 0.32
Neu for User I15: 0.37
Rep for User I16: 0.37
Dem for User I16: 0.00
Neu for User I16: 0.29
Rep for User I17: 0.35
Dem for User I17: 0.00
Neu for User I17: 0.35
Rep for User I18: 0.35
Dem for User I18: 0.00
Neu for User I18: 0.35
Rep for User I19: 0.35
Dem for User I19: 0.00
Neu for User I19: 0.35
Rep for User I20: 0.35
Dem for User I20: 0.00
Neu for User I20: 0.35
Rep for User I21: 0.28
Dem for User I21: 0.32
Neu for User I21: 0.36


### Normalized topic entropy

- Original Normalized topic entropy: similar to normalized stance entropy, they measure the
diversity of topics. This provides a measure of topical diversity, in addition to stance diversity above. The metric is the same as Equation 3, where pi is instead the probability of articles having topic i in a sequence of recommendations, and m = 14 since there are 14 topics.

- Our Normalized topic entropy: c*alculate_normalized_topic_entropy* takes the matrix and the value of m (in your case, 15, as 15 total topics) as input and **calculates the normalized topic entropy based on the fractions of articles that have each topic in a recommendation sequence**.

In [11]:
# Definisci una funzione per calcolare l'entropia dell'argomento normalizzata
def calculate_normalized_topic_entropy(matrix, m):
    # Calcola le frazioni di articoli che hanno ciascun argomento in una sequenza di raccomandazioni
    topic_fractions = np.sum(matrix, axis=0) / np.sum(matrix)
    
    # Calcola l'entropia della distribuzione
    entropy = -np.sum(topic_fractions * np.log(topic_fractions)) / np.log(m)
    
    return entropy

# Calcola l'entropia dell'argomento normalizzata con m = 15
m = 15
normalized_topic_entropy = calculate_normalized_topic_entropy(matrix, m)
print("Normalized Topic Entropy:", normalized_topic_entropy)

Normalized Topic Entropy: 0.8105601210727017


This metric represents the diversity of article topics recommended to users. A higher entropy value indicates a greater diversity of recommended content topics. In this case, a value of 0.8105601210727017 suggests some diversity in recommended article topics.